In [ ]:
# | default_exp _components.new_project_generator

In [ ]:
# | export


from typing import *
import shutil
from pathlib import Path
import os
from tempfile import TemporaryDirectory

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.helper import (
    download_and_extract_faststream_archive,
    write_file_contents,
    read_file_contents
)
from faststream_gen._code_generator.constants import (
    FASTSTREAM_TEMPLATE_ZIP_URL,
    FASTSTREAM_TEMPLATE_DIR_SUFFIX,
    INTERMEDIATE_RESULTS_DIR_NAME,
    APPLICATION_FILE_NAME,
    INTEGRATION_TEST_FILE_NAME
)

In [ ]:


import pytest

from faststream_gen._components.logger import suppress_timestamps
from faststream_gen._code_generator.constants import DESCRIPTION_FILE_NAME

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def create_project(output_path: str, save_intermediate_files: bool) -> None:
     with yaspin(
        text="Creating a new FastStream project...", color="cyan", spinner="clock"
    ) as sp:
            
        with download_and_extract_faststream_archive(FASTSTREAM_TEMPLATE_ZIP_URL) as extracted_path:
            with TemporaryDirectory() as tmp_dir:
                app_path = f"{tmp_dir}/app/application.py"
                test_path = f"{tmp_dir}/tests/test_application.py"
                intermediate_dir_path = f"{output_path}/{INTERMEDIATE_RESULTS_DIR_NAME}"
                shutil.copytree(str(extracted_path / FASTSTREAM_TEMPLATE_DIR_SUFFIX), tmp_dir, dirs_exist_ok=True)
                shutil.copy(f"{intermediate_dir_path}/{APPLICATION_FILE_NAME}", app_path)
                shutil.copy(f"{intermediate_dir_path}/{INTEGRATION_TEST_FILE_NAME}", test_path)

                test_file_contents = read_file_contents(test_path)
                test_file_contents = test_file_contents.replace("from application import", "from app.application import")
                write_file_contents(test_path, test_file_contents)

                shutil.copytree(tmp_dir, output_path, dirs_exist_ok=True)
                if not save_intermediate_files:
                    shutil.rmtree(intermediate_dir_path)
                
        sp.text = ""
        sp.ok(f" ✔ New FastStream project created.")

In [ ]:
fixture_application_code = """
message = "hi"
print(message)
"""

fixture_test_code = """
from application import message
"""

fixture_description = """
description
"""

for flag in [True, False]:
    with TemporaryDirectory() as d:
        intermediate_results_dir = Path(d)/INTERMEDIATE_RESULTS_DIR_NAME
        intermediate_results_dir.mkdir(parents=True, exist_ok=True)

        app_file_name = intermediate_results_dir / APPLICATION_FILE_NAME
        write_file_contents(str(app_file_name), fixture_application_code)

        test_file_name = intermediate_results_dir / INTEGRATION_TEST_FILE_NAME
        write_file_contents(str(test_file_name), fixture_test_code)

        description_file_name = intermediate_results_dir / DESCRIPTION_FILE_NAME
        write_file_contents(str(description_file_name), fixture_description)

        create_project(d, flag)
        files = [p.stem for p in list(Path(f"{d}").glob("*"))]
        print(files)
        if flag:
            assert INTERMEDIATE_RESULTS_DIR_NAME in files
        else:
            assert INTERMEDIATE_RESULTS_DIR_NAME not in files
        assert "README" in files

⠹ Creating a new FastStream project... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✔ New FastStream project created.     
['README', '.github', 'LICENSE', '_faststream_gen_tmp', 'tests', 'app', 'requirements', '.gitignore']
 ✔ New FastStream project created.     
['README', '.github', 'LICENSE', 'tests', 'app', 'requirements', '.gitignore']
